In [51]:
# =============================================
# 1. CONFIGURACIÓN INICIAL
# =============================================

# 1️⃣ Importaciones
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import spearmanr
import pandas as pd
import numpy as np
import sqlite3

# Ruta al DW
db_path = "C:/Users/Nico/Desktop/DATA SCIENCE/PP- VOLUNTAREADO/chivas-ml/data/external/chivas_dw.sqlite"

# Conexión
conn = sqlite3.connect(db_path)

# Cargar la vista combinada
df = pd.read_sql_query("SELECT * FROM vw_pred_diaria_training_ce", conn)
# Cerrar conexión
conn.close()

print(f"Registros cargados: {len(df)}")
df.head()

Registros cargados: 2216


,id_jugador,microciclo_actual,CT_total_actual,CE_total_actual,CS_total_actual,CR_total_actual,riesgo_suavizado_3d_actual,entrenos_total_next,descansos_total_next,partidos_total_next,...,Edad,Peso_kg,Estatura_cm,Posicion,Linea,Fecha,tipo_dia_next,target_CE_diaria,target_CS_diaria,target_Dist_diaria
0,1,7,0.0,0.0,0.0,0.0,0.00,1,6,0,...,25,87.8,191.0,Arquero,Portería,2025-07-08,2.0,213.5,209.0,4587
1,1,17,2879.7,160.7,203.0,2516.0,0.00,1,4,2,...,25,87.8,191.0,Arquero,Portería,2025-09-15,1.0,87.0,41.8,1645
2,1,19,6191.0,203.7,337.8,5649.5,0.01,4,2,1,...,25,87.8,191.0,Arquero,Portería,2025-10-01,2.0,174.8,176.2,3207
3,1,19,6191.0,203.7,337.8,5649.5,0.01,4,2,1,...,25,87.8,191.0,Arquero,Portería,2025-10-02,3.0,187.0,154.2,3666
4,1,19,6191.0,203.7,337.8,5649.5,0.01,4,2,1,...,25,87.8,191.0,Arquero,Portería,2025-10-03,-2.0,145.0,64.2,2097


In [52]:
# Primero, como hicimos en todos los dataset a entrenar, excluimos todos los jugadores 
# que no proporcionan valor, al igual que los microciclos donde no se compitió:

# =============================================
# 2.1 FILTRO DE JUGADORES Y MICRO-CICLOS
# =============================================

print(f"Registros cargados pre filtrado: {len(df)}\n")

# Jugadores que deben excluirse
jugadores_excluir = [1, 2, 3, 12, 30]

# Filtrar jugadores no válidos
df = df[~df["id_jugador"].isin(jugadores_excluir)]

# fFiltramos los días nulos o no validos (elimina microciclos fuera de competencia):
df = df[df['tipo_dia_next'].notnull()]


print(f"Registros cargados post filtrado: {len(df)}\n")
# Distribución de la etiqueta
print("\nDistribución de tipo_semana_next:")
print(df['tipo_semana_next'].value_counts(normalize=True) * 100)

Registros cargados pre filtrado: 2216

Registros cargados post filtrado: 1445


Distribución de tipo_semana_next:
tipo_semana_next
CARGA       46.989619
RECUPERO    32.941176
MANTENER    20.069204
Name: proportion, dtype: float64


In [53]:
print(df['tipo_semana_next'].value_counts(normalize=True) * 100)
print(df['tipo_dia_next'].value_counts())


tipo_semana_next
CARGA       46.989619
RECUPERO    32.941176
MANTENER    20.069204
Name: proportion, dtype: float64
tipo_dia_next
-1.0    467
 1.0    316
 2.0    273
-2.0    255
 3.0    134
Name: count, dtype: int64


In [58]:
import pandas as pd
import numpy as np

# Supongamos que ya cargaste el df desde vw_pred_diaria_training_ce
# df = pd.read_sql_query("SELECT * FROM vw_pred_diaria_training_ce", conn)

# 🔹 Calcular totales semanales reales (por microciclo_ref = semana actual)
totales = (
    df.groupby(['id_jugador', 'microciclo_actual'])
    [['target_CE_diaria', 'target_CS_diaria', 'target_Dist_diaria']]
    .sum()
    .rename(columns={
        'target_CE_diaria': 'CE_total_sem_real',
        'target_CS_diaria': 'CS_total_sem_real',
        'target_Dist_diaria': 'Dist_total_sem_real'
    })
    .reset_index()
)

# 🔹 Unir estos totales al dataframe original
df = df.merge(totales, on=['id_jugador', 'microciclo_actual'], how='left')

# 🔹 Calcular porcentaje relativo que representa cada día dentro de su microciclo
df['CE_pct_microciclo'] = (df['target_CE_diaria'] / df['CE_total_sem_real']) * 100
df['CS_pct_microciclo'] = (df['target_CS_diaria'] / df['CS_total_sem_real']) * 100
df['Dist_pct_microciclo'] = (df['target_Dist_diaria'] / df['Dist_total_sem_real']) * 100

# 🔹 Limpiar valores nulos o infinitos
df[['CE_pct_microciclo', 'CS_pct_microciclo', 'Dist_pct_microciclo']] = df[
    ['CE_pct_microciclo', 'CS_pct_microciclo', 'Dist_pct_microciclo']
].fillna(0).replace([np.inf, -np.inf], 0)

print(df[['Fecha', 'tipo_dia_next', 'CE_pct_microciclo', 'CS_pct_microciclo', 'Dist_pct_microciclo']].head())



        Fecha  tipo_dia_next  CE_pct_microciclo  CS_pct_microciclo  \
0  2025-07-07            1.0          16.829533          18.662567   
1  2025-07-08            2.0          23.835151          24.109659   
2  2025-07-09            3.0          36.543455          32.072765   
3  2025-07-10           -2.0           2.832460           1.019728   
4  2025-07-11           -1.0          19.959401          24.135281   

   Dist_pct_microciclo  
0            17.498897  
1            22.747097  
2            30.264125  
3             6.924095  
4            22.565786  


In [59]:
# Features
FEATURES = [
    'entrenos_total_next', 'descansos_total_next', 'partidos_total_next',
    'entrenos_pre_partido_next', 'entrenos_post_partido_next',
    'CE_total_next', 'CT_total_actual',
    'tipo_dia_next', 'riesgo_suavizado_3d_actual',
    'Edad', 'Peso_kg', 'Estatura_cm'
] + [c for c in df.columns if c.startswith('Pos_') or c.startswith('Lin_')]


X = df[FEATURES]

# 🎯 Target diario de la semana siguiente
y = df['CE_pct_microciclo']


# Limpieza por si hubiera NaN
mask = ~y.isna()
X, y = X[mask], y[mask]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# (Opcional) RF no requiere escalado, pero si querés dejarlo:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

model = RandomForestRegressor(
    n_estimators=500,
    max_depth=16,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)
print("R²:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
rho, _ = spearmanr(y_test, y_pred)
print("Spearman ρ:", rho)

R²: 0.4165054115451944
MAE: 10.1025468184665
Spearman ρ: 0.5799870614840561


In [60]:
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.stats import spearmanr

# Agregar las predicciones al dataframe de test
df_eval = X_test.copy()
df_eval['Real'] = y_test.values
df_eval['Predicho'] = y_pred
df_eval['tipo_dia_next'] = df.loc[y_test.index, 'tipo_dia_next'].values  # Aseguramos alineación

# Evaluación por intensidad
resultados_por_intensidad = []
for intensidad in sorted(df_eval['tipo_dia_next'].unique()):
    subset = df_eval[df_eval['tipo_dia_next'] == intensidad]
    if len(subset) < 5:  # evitamos divisiones con muy pocos casos
        continue
    
    r2 = r2_score(subset['Real'], subset['Predicho'])
    mae = mean_absolute_error(subset['Real'], subset['Predicho'])
    rho, _ = spearmanr(subset['Real'], subset['Predicho'])
    
    resultados_por_intensidad.append({
        'Intensidad': intensidad,
        'R²': round(r2, 3),
        'MAE': round(mae, 3),
        'ρ (Spearman)': round(rho, 3),
        'N muestras': len(subset)
    })

df_resultados = pd.DataFrame(resultados_por_intensidad)
print("📊 Resultados por Intensidad:")
display(df_resultados)


📊 Resultados por Intensidad:


,Intensidad,R²,MAE,ρ (Spearman),N muestras
0,-2.0,0.484,8.756,0.600,51
1,-1.0,0.572,8.204,0.618,85
2,1.0,0.254,13.117,0.287,71
3,2.0,0.143,9.883,0.350,59
4,3.0,-0.057,11.363,0.007,23


Vamor a probar un sistema jerárquico de pdicciones, según los resultados que arojó el gráfico por intensidad. Se prebó entrenar 